In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from datetime import datetime

model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

In [5]:
import sys
sys.path.append('./models')


from main_model import *
import sys 
sys.path.append('/data/users2/rohib/github/testing')
import utils_gsp.sps_tools as sps_tools
import utils_gsp.gpu_projection as gsp_gpu

import sys
sys.path.append('./models')
# import models.resnet_torch as ResNet
import dali_loader.dali_loaders as Dali

In [6]:
class Args:
    data = '/data/users2/rohib/github/imagenet-data'
    data_backend = 'dali-cpu'
    interpolation = 'bilinear'
    image_size = 224
    mixup = 0.0
    num_classes = 1000
    augmentation = None
    memory_format = "nchw"
    arch = 'resnet18'
    workers = 4
    epochs = 1
    start_epoch = 0
    batch_size = 16
    lr = 0.1
    momentum = 0.9
    weight_decay = 1e-4
    print_freq = 10
    resume = ''
    evaluate = False
    pretrained = False
    world_size = -1
    dist_url = 'tcp://224.66.41.62:23456'
    dist_backend = 'nccl'
    seed = None
    gpu = None
    multiprocessing_distributed = False

args = Args

In [10]:
train_loader, train_loader_len, val_loader, val_loader_len = Dali.get_dali_loaders(args)

/home/users/rohib/anaconda3/envs/nv-imagenet/lib/python3.8/site-packages/nvidia/dali/ops.py:627: DeprecationWarning: WARNING: `coin_flip` is now deprecated. Use `random.coin_flip` instead.
  op_instances.append(_OperatorInstance(input_set, self, **kwargs))
/home/users/rohib/anaconda3/envs/nv-imagenet/lib/python3.8/site-packages/nvidia/dali/ops.py:627: DeprecationWarning: WARNING: `file_reader` is now deprecated. Use `readers.file` instead.
In DALI 1.0 all readers were moved into a dedicated :mod:`~nvidia.dali.fn.readers`
submodule and renamed to follow a common pattern. This is a placeholder operator with identical
functionality to allow for backward compatibility.
  op_instances.append(_OperatorInstance(input_set, self, **kwargs))
/home/users/rohib/anaconda3/envs/nv-imagenet/lib/python3.8/site-packages/nvidia/dali/ops.py:627: DeprecationWarning: WARNING: `image_decoder` is now deprecated. Use `decoders.image` instead.
In DALI 1.0 all decoders were moved into a dedicated :mod:`~nvidia.

In [5]:
args.multiprocessing_distributed

False

In [8]:
from dali_loader.dataloaders import *


In [7]:
def get_model(args):
    if args.seed is not None:
        random.seed(args.seed)
        torch.manual_seed(args.seed)
        cudnn.deterministic = True
        warnings.warn('You have chosen to seed training. '
                        'This will turn on the CUDNN deterministic setting, '
                        'which can slow down your training considerably! '
                        'You may see unexpected behavior when restarting '
                        'from checkpoints.')

    if args.gpu is not None:
        warnings.warn('You have chosen a specific GPU. This will completely '
                        'disable data parallelism.')

    if args.dist_url == "env://" and args.world_size == -1:
        args.world_size = int(os.environ["WORLD_SIZE"])

    args.distributed = args.world_size > 1 or args.multiprocessing_distributed

    ngpus_per_node = torch.cuda.device_count()
    if args.multiprocessing_distributed:
        # Since we have ngpus_per_node processes per node, the total world_size
        # needs to be adjusted accordingly
        args.world_size = ngpus_per_node * args.world_size
        # Use torch.multiprocessing.spawn to launch distributed processes: the
        # main_worker process function
        mp.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, args))
    else:
        # Simply call main_worker function
        model, train_loader = main_worker(args.gpu, ngpus_per_node, args)
    return model, train_loader

model, train_loader = get_model(args)

<ipython-input-7-9c72efcadaf6>:13: UserWarning: You have chosen a specific GPU. This will completely disable data parallelism.
  warnings.warn('You have chosen a specific GPU. This will completely '


Use GPU: 0 for training
=> creating model 'resnet18'
Created model from local file!


In [32]:
# memory_format = (
#     torch.channels_last if args.memory_format == "nhwc" else torch.contiguous_format
# )
# image_size = (
#     args.image_size
#     if args.image_size is not None
#     else model.arch.default_image_size
# )
# # Create data loaders and optimizers as needed
# if args.data_backend == "pytorch":
#     get_train_loader = get_pytorch_train_loader
#     get_val_loader = get_pytorch_val_loader
# elif args.data_backend == "dali-gpu":
#     get_train_loader = get_dali_train_loader(dali_cpu=False)
#     get_val_loader = get_dali_val_loader()
# elif args.data_backend == "dali-cpu":
#     get_train_loader = get_dali_train_loader(dali_cpu=True)
#     get_val_loader = get_dali_val_loader()
# elif args.data_backend == "syntetic":
#     get_val_loader = get_syntetic_loader
#     get_train_loader = get_syntetic_loader
# else:
#     print("Bad databackend picked")
#     exit(1)

In [34]:
# train_loader, train_loader_len = get_train_loader(
#     args.data,
#     image_size,
#     args.batch_size,
#     args.num_classes,
#     args.mixup > 0.0,
#     interpolation = args.interpolation,
#     augmentation=args.augmentation,
#     start_epoch=start_epoch,
#     workers=args.workers,
#     memory_format=memory_format,
# )
# if args.mixup != 0.0:
#     train_loader = MixUpWrapper(args.mixup, train_loader)

# val_loader, val_loader_len = get_val_loader(
#     args.data,
#     image_size,
#     args.batch_size,
#     args.num_classes,
#     False,
#     interpolation = args.interpolation,
#     workers=args.workers,
#     memory_format=memory_format,
# )

/home/users/rohib/anaconda3/envs/nv-imagenet/lib/python3.8/site-packages/nvidia/dali/ops.py:627: DeprecationWarning: WARNING: `coin_flip` is now deprecated. Use `random.coin_flip` instead.
  op_instances.append(_OperatorInstance(input_set, self, **kwargs))
/home/users/rohib/anaconda3/envs/nv-imagenet/lib/python3.8/site-packages/nvidia/dali/ops.py:627: DeprecationWarning: WARNING: `file_reader` is now deprecated. Use `readers.file` instead.
In DALI 1.0 all readers were moved into a dedicated :mod:`~nvidia.dali.fn.readers`
submodule and renamed to follow a common pattern. This is a placeholder operator with identical
functionality to allow for backward compatibility.
  op_instances.append(_OperatorInstance(input_set, self, **kwargs))
/home/users/rohib/anaconda3/envs/nv-imagenet/lib/python3.8/site-packages/nvidia/dali/ops.py:627: DeprecationWarning: WARNING: `image_decoder` is now deprecated. Use `decoders.image` instead.
In DALI 1.0 all decoders were moved into a dedicated :mod:`~nvidia.

---------------

In [15]:
import utils_gsp.gpu_projection as gsp_gpu
gsp_func = gsp_gpu
sps = 0.8

In [16]:
model.curr_iter = 0 
model.every_steps = 4

def forward_pre_hook_gsp(m, x):
    if (self.curr_iter % self.every_steps) == 0:
        print(f"Applying GSP Hook at step: {self.curr_iter} on layer: {m.name} | device: {torch.cuda.current_device()}")
        apply_hook_gsp(m, sps=0.8)
    else:
        pass
        print(f"Current Step: {self.curr_iter} from device: {torch.cuda.current_device()}")
    # print(f"Layer: current iter: {curr_iter}")
    self.steps += 1

In [17]:
# class GSP_Hook():
#     def __init__(self, name, every_steps=1):
#         self.name = name
#         self.every_steps = every_steps
#         self.steps = 0
    
#     # def hook(self, model, input, output):
#     #     if (self.steps % self.every_steps) == 0:
#     #         self.activation[self.name] = output.detach()
#     #     self.steps += 1
    
#     def forward_pre_hook_gsp(self, m, x):
#         if (self.steps % self.every_steps) == 0:
#             print(f"Applying GSP Hook at step: {self.steps} on layer: {m.name} | device: {torch.cuda.current_device()}")
#             apply_hook_gsp(m, sps=0.8)
#         else:
#             pass
#             print(f"Current Step: {self.steps} from device: {torch.cuda.current_device()}")
#         print(f"Layer: current iter: {curr_iter}")
#         self.steps += 1

In [18]:
model.curr_iter = 0

sps_tools.get_abs_sps(model)
for name, layer in model.named_modules():
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
        layer.name = name
        layer.curr_iter = model.curr_iter
        # gsp_hook = GSP_Hook(name, every_steps=4)
        # layer.register_forward_pre_hook(gsp_hook.forward_pre_hook_gsp)
        layer.register_forward_pre_hook(forward_pre_hook_gsp)

In [19]:
def apply_hook_gsp(layer, sps):
    count =0
    layer_d = {}
    gsp_in_d = {}

    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
        # layer_d[name] = layer
        w_shape = layer.weight.shape

        if 'downsample' in layer.name:
            # print(layer.weight.shape)
            dim_1 ,  dim_2 = layer.weight.shape[0], layer.weight.shape[1]
            gsp_in_d[name] = layer.weight.detach().reshape(dim_1, -1)
        else:
            # print(layer.weight.shape[0])
            gsp_in_d[name] = layer.weight.detach().reshape(layer.weight.shape[0], -1)
        
        layer.weight.data = gsp_gpu.groupedsparseproj(gsp_in_d[name].T, sps=0.8).T.reshape(w_shape)
    
    # del gsp_in_d

In [17]:
model.sps = 0.9

In [7]:
model.curr_epoch = 1

In [14]:
print(model.curr_iter)
print(model.curr_epoch)

images, target = next(iter(train_loader))
images = images.cuda(args.gpu, non_blocking=True)
target = target.cuda(args.gpu, non_blocking=True)
output = model(images)

model.curr_iter += 1

4
1
Current iter: 4
Module | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Layer | requires_grad: True
Module | requires_grad: True
GSP applied to all layers!


In [13]:
sps_tools.get_layerwise_sps(model)
# sps_tools.get_model_layers(model)

{'conv1.weight': tensor(0.7871, device='cuda:0'),
 'bn1.weight': tensor(0., device='cuda:0'),
 'layer1.0.conv1.weight': tensor(0.7782, device='cuda:0'),
 'layer1.0.bn1.weight': tensor(0., device='cuda:0'),
 'layer1.0.conv2.weight': tensor(0.7776, device='cuda:0'),
 'layer1.0.bn2.weight': tensor(0., device='cuda:0'),
 'layer1.1.conv1.weight': tensor(0.7780, device='cuda:0'),
 'layer1.1.bn1.weight': tensor(0., device='cuda:0'),
 'layer1.1.conv2.weight': tensor(0.7783, device='cuda:0'),
 'layer1.1.bn2.weight': tensor(0., device='cuda:0'),
 'layer2.0.conv1.weight': tensor(0.7746, device='cuda:0'),
 'layer2.0.bn1.weight': tensor(0., device='cuda:0'),
 'layer2.0.conv2.weight': tensor(0.7822, device='cuda:0'),
 'layer2.0.bn2.weight': tensor(0., device='cuda:0'),
 'layer2.0.downsample.0.weight': tensor(0.7049, device='cuda:0'),
 'layer2.0.downsample.1.weight': tensor(0., device='cuda:0'),
 'layer2.1.conv1.weight': tensor(0.7823, device='cuda:0'),
 'layer2.1.bn1.weight': tensor(0., device='cuda

In [25]:
# # print(model.curr_iter)
# # compute output
# # output = model(images)
# # loss = criterion(output, target)

count = 0
for name, layer in model.named_modules():
    # layer_d[name] = layer
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
        # print(name)
        count +=1
print(count)

21


In [ ]:
# compute output
output = model(images)
loss = criterion(output, target)

# measure accuracy and record loss
acc1, acc5 = accuracy(output, target, topk=(1, 5))
losses.update(loss.item(), images.size(0))
top1.update(acc1[0], images.size(0))
top5.update(acc5[0], images.size(0))

# compute gradient and do SGD step
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [ ]:
count = 0
for name, layer in model.named_modules():
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
        count+=1
print(count)

In [ ]:
sps_tools.get_abs_sps(model)
# model, train_loader = get_model(args)
model

In [ ]:
count =0
orig_shape={}
layer_d = {}
gsp_in_d = {}
post_gsp_shp = {}

for name, layer in model.named_modules():
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
        layer_d[name] = layer
        orig_shape[name] = layer.weight.shape
        w_shape = layer.weight.shape
        
        if 'downsample' in name:
            # print(layer.weight.shape)
            dim_1 ,  dim_2 = layer.weight.shape[0], layer.weight.shape[1]
            gsp_in_d[name] = layer.weight.detach().reshape(dim_1, -1)
        else:
            # print(layer.weight.shape[0])
            gsp_in_d[name] = layer.weight.detach().reshape(layer.weight.shape[0], -1)
        
        # print(gsp_in_d[name].shape)
        # sparse_l = gsp_gpu.groupedsparseproj(gsp_in_d[name].T, sps=0.8).T
        # print(sparse_l.shape)
        layer.weight.data = gsp_gpu.groupedsparseproj(gsp_in_d[name].T, sps=0.8).T.reshape(w_shape)
        post_gsp_shp[name] = layer.weight.shape
        
# gsp_shape_d = [x.shape for x in gsp_in_d.values()]
# list(gsp_shape_d)

In [ ]:
# gsp_shape_d = [x.shape for x in gsp_in_d.values()]
# list(post_gsp_shp.values())

In [ ]:
print(layer_d['module.layer2.0.downsample.0'].weight.shape)
print(layer_d['module.layer3.0.downsample.0'].weight.reshape(128, -1).shape)
# list(gsp_in_d.keys())
# gsp_shape_d = [x.shape for x in gsp_in_d.values()]

# list(gsp_shape_d)



In [ ]:
print(layer.weight.shape)
sparse_weight.shape

In [ ]:
print(f"The total model sparsity is: {sps_tools.get_abs_sps(model)[0].item()}")
sps_tools.gsp_conv_linear(model, sps=0.9)
print(f"The total model sparsity after GSP: {sps_tools.get_abs_sps(model)[0].item()}")

In [12]:
model_test = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          nn.Conv2d(20,64,5),
          nn.ReLU()
        )

In [13]:
for name, param in model_test.named_parameters():
    print(name)

0.weight
0.bias
2.weight
2.bias
